## ## Table of Contents
- [Introduction](#intro)
- [Part I - Gathering Data](#Gathering_Data)
> - [*csv file](#csv)
> - [*tsv file](#tsv)
> - [*Twitter API](#tweet_api)
- [Part II - Assess Data](#Assesing_Data)
- [Part III - Clean Data](#Clean_Data)
- [Part IV - Analysis](#Analysis)
- [Part V - Conclusions summary](#conclusions)


<a id='intro'></a>
### Introduction

The project aimed to analyze the data in <a href = 'https://twitter.com/dog_rates'>"WeRateDogs"</a> channel on Twitter.com.<br>

#### Gathering Data
First, we gather data from different sources:
- **\*.csv file, stored in the local machine.** This file contains preliminarily gathered data of tweets specified by a tweeet_id parameter. The file should be assessed, cleaned and structured in a desired manner.
- **\*tsv file stored in the remote server.** This file contains the data about breed prediction based on pictures gathered from tweets in the channel. Each tweet specified by the tweet_id parameter. The file needs to be assessed, cleaned and structured in the desired way.
- **Twitter API** - an archive of all tweets in the channel. We need to get additional information from the Twitter server. Since we have the tweet_id parameter in both files mentioned above, we going to extract particular data, associated with each tweet_id from the server.<br><br>

#### Assess and Clean Data

We have three different sources of data and we need to get one solid data set with good quality and tidy data. Perform assessing data from each source and fix every issue we found to get data ready for analysis.


#### Analysis




- Whether or not retweet count depends on the breed? If yes - what the top three breeds in retweets?
- Whether or not favorite count depends on the breed?  If yes - what the top three breeds in favorites?
- Is there most popular dog names and what are they if exist?



<a id='Gathering_Data'></a>
# Part I - Gathering Data

To get started, let's import libraries.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import matplotlib.pyplot as plt


<br>
<a id='csv'></a>

###  Gathering data from *.csv file

In [15]:
#read the cav file and store into pandas data frame
df_master = pd.read_csv('twitter-archive-enhanced.csv')

In [16]:
#visual assessmnet is good with no ".head(1)"
df_master.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


<br>
<a id='tsv'></a>

###  Gathering data from *.tsv file

In [17]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
#create an object with data we got from url request
r = requests.get(url)


In [18]:
# create file in home directory and write the content of object into it
with open('predict.tsv', 'wb') as f:
    f.write(r.content)

In [19]:
#read file and import data into the pandas df
predict = pd.read_csv('predict.tsv', sep = '\t')

In [20]:
predict.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


<br>
<a id='tweet_api'></a>

###  Gathering data from Tweitter API
The idea of this part is to get retweet count and favorite count for each tweet_id we have in 'predict' data frame to associate that counts with a particular breed later
To get started, let's import libraries

In [2]:
import tweepy
import json

In [123]:
# get a list of tweets id
id_list = df['tweet_id'].tolist();
# get the length of id list
len(id_list)

2299

In [124]:
#check
id_list[0:3]

[892420643555336193, 892177421306343426, 891815181378084864]

In [161]:
#create connections and API object for further use 
from key import key
consumer_key = key['consumer_key']
consumer_secret = key['consumer_secret']
access_token = key['access_token']
access_secret = key['access_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#pass auth and set the True values for the time-limit parameters of API-class object
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [125]:
import time

#Get API objects for each id, check if such id doesn't exist, write json data into the file

count = 0 #counter of itterations
the_begining = time.time() # set the timer on start
no_id_list = {} #dict for storing not existing ids

with open ('tweet_json.txt', 'w+') as file: #create and open a file
    for id in id_list: #loop through the list of ids
        
        try: #attempts to do thing with a possibility to catch errors
            
            #count +=1
            #print(str(count),'-', id)   
            tweet = api.get_status(id, tweet_mode='extended', include_entities = True); #create API object for specified id
            #print('Hurrrraaaay!') 
            
            json.dump(tweet._json, file)#write the json data from API object to the file
            file.write('\n')# take a new line for the next entry 
            
        except tweepy.TweepError as e:
                #print('Fail')
                no_id_list[id] = e #assign the error value to the failed id, store it in dict
                pass
            
the_end = time.time() #set the timer on finish
print('Session duration is', ' ', the_end - the_begining)
print('Fails dump', '\n', no_id_list)

Rate limit reached. Sleeping for: 454
Rate limit reached. Sleeping for: 597


Session duration is   1847.1826527118683
Fails dump 
 {888202515573088257: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 873697596434513921: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 872261713294495745: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 869988702071779329: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 866816280283807744: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 861769973181624320: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 845459076796616705: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 842892208864923648: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 827228250799742977: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 812747805718642688: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 802247111496568

In [126]:
# Print failed ids
for key in no_id_list:
    print(key)

888202515573088257
873697596434513921
872261713294495745
869988702071779329
866816280283807744
861769973181624320
845459076796616705
842892208864923648
827228250799742977
812747805718642688
802247111496568832
775096608509886464
770743923962707968
754011816964026368
680055455951884288


Let's read our 'text_json' file. <br>
Get just three elements we need: 'Tweet id', 'Expanded URL', 'Retweet Count' and 'Favorite Count' for each id. Store these data into the new data frame


In [4]:
#create new data frame with desired columns names
counts_df = pd.DataFrame(columns = ['tweet_id', 'entities', 'retweet_count', 'favor_count'])

with open ('tweet_json.txt') as file: #open file
#read file line by line, extract data, store it in dataframe object and append it to counts_df data frame
    for line in file:
        status = json.loads(line) #get a json object out of each line in a file
        tweet_id = status['id_str'] #get id element
        retweet_count = status['retweet_count']# get retweets count
        favor_count = status['favorite_count']#get favorites count
        entities = status['entities']
        counts_df = counts_df.append(pd.DataFrame([[tweet_id, entities, retweet_count, favor_count]], \
                                                 columns = ['tweet_id', 'entities', 'retweet_count', 'favor_count']))
counts_df = counts_df.reset_index(drop = True)

In [5]:
counts_df.head(2)

,tweet_id,entities,retweet_count,favor_count
0,892420643555336193,"{'hashtags': [], 'symbols': [], 'user_mentions...",8238,37795
1,892177421306343426,"{'hashtags': [], 'symbols': [], 'user_mentions...",6088,32470


In [6]:
def extract_url(dict):
    if 'media' in dict:
        return dict['media'][0]['expanded_url']
    else:
        return None

In [7]:
# get expanded_url variable values
counts_df['expanded_url'] = [extract_url(entities) for entities in counts_df['entities']]

In [8]:
counts_df.head(2)

,tweet_id,entities,retweet_count,favor_count,expanded_url
0,892420643555336193,"{'hashtags': [], 'symbols': [], 'user_mentions...",8238,37795,https://twitter.com/dog_rates/status/892420643...
1,892177421306343426,"{'hashtags': [], 'symbols': [], 'user_mentions...",6088,32470,https://twitter.com/dog_rates/status/892177421...


In [9]:
#drop 'entities' column
counts_df.drop('entities', axis = 1, inplace = True)

In [10]:
counts_df.shape[0]

2284

In [11]:
counts_df['expanded_url'].isnull().sum()

232

In [42]:
counts_df[counts_df['expanded_url'].isnull() == True].sample(5)

,tweet_id,retweet_count,favor_count,expanded_url
961,746542875601690625,1985,5251,None
1871,673716320723169284,776,3125,None
606,791774931465953280,24204,48046,None
345,826598799820865537,270,5374,None
1675,679405845277462528,1270,2465,None


In [13]:
counts_df[counts_df['expanded_url'].str.contains('dog_rates') == False].shape[0]

0

In [43]:
counts_df.to_csv('counts_df.csv', index = False)

<br><br>**No we have three different data frames:**
- "df_master" - the data frame with a bunch of data associated with tweet_id. Gathered from *.csv file
- "predict" - the data frame with dog breed prediction data associated with the picture, captured from the tweet with specified twee_id. Gathered from *.tsv file
- "counts_df" - the data frame with data about retweet_counts and favorite_count associated with particular tweet_id. Gathered from through Twitter API
<br><br>
Let's assess each data frame for the quality and tidy issues

<br>
<a id='Assesing_Data'></a>

# Part II - Assess Data
  
    

In [21]:
# copy data frame
df = df_master.copy()

In [22]:
# visual assessmnet is ok with no ".head(1)"
# additional vs is ok in Excel
df.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [23]:
df.shape

(2356, 17)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [25]:
df.nunique()

tweet_id                      2356
in_reply_to_status_id           77
in_reply_to_user_id             31
timestamp                     2356
source                           4
text                          2356
retweeted_status_id            181
retweeted_status_user_id        25
retweeted_status_timestamp     181
expanded_urls                 2218
rating_numerator                40
rating_denominator              18
name                           957
doggo                            2
floofer                          2
pupper                           2
puppo                            2
dtype: int64

In [26]:
df.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [27]:
df.text[9]

'This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A'

In [28]:
df.text[19]

'RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX'

In [29]:
df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [30]:
pr = predict.copy()

In [31]:
pr.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [32]:
pr.shape

(2075, 12)

In [33]:
pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [34]:
pr.nunique()

tweet_id    2075
jpg_url     2009
img_num        4
p1           378
p1_conf     2006
p1_dog         2
p2           405
p2_conf     2004
p2_dog         2
p3           408
p3_conf     2006
p3_dog         2
dtype: int64

In [35]:
count = counts_df.copy()

In [36]:
count.head()

,tweet_id,retweet_count,favor_count,expanded_url
0,892420643555336193,8238,37795,https://twitter.com/dog_rates/status/892420643...
1,892177421306343426,6088,32470,https://twitter.com/dog_rates/status/892177421...
2,891815181378084864,4029,24443,https://twitter.com/dog_rates/status/891815181...
3,891689557279858688,8395,41123,https://twitter.com/dog_rates/status/891689557...
4,891327558926688256,9091,39333,https://twitter.com/dog_rates/status/891327558...


In [37]:
count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2284 entries, 0 to 2283
Data columns (total 4 columns):
tweet_id         2284 non-null object
retweet_count    2284 non-null object
favor_count      2284 non-null object
expanded_url     2052 non-null object
dtypes: object(4)
memory usage: 71.5+ KB


In [38]:
count.nunique()

tweet_id         2284
retweet_count    1689
favor_count      1977
expanded_url     1991
dtype: int64

# <br>

## Assessing Report

<br>

### Quality
**df**
- Columns 'in_reply_to_status_id' and 'in_reply_to_user_id' has very little amount of data and make no sense for further analysis. Drop'm
- Data in every cell in column 'timestamp' has '+0000' at the end of the string. 
- Column 'timestamp' in 'object' data type. Convert to 'datetime'
- Column 'source' has wrong data totally. Drop it
- Some cells in the column 'text' contains '@' sign with some diffirent chanales names, that associated with retweets. Need to get closer look and figure it out.
- Columns 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' are lack of data. Drop'm
- Some cells in column 'expanded_urls' has two or more urls and some of urls doesn't belong to WeRateDogs chanel.
- Duplicated and missing data in column 'expanded_urls' 
- Column 'rating_numerator' has several outliers, so need to get closer what are they exactly.
- Column 'rating_denominator' - need to figure out the outliers.
- Column 'name' has some wrong data such as 'None' and other words that not suppose to be the names of dogs. Need to figure out how to handle them.
- Columns 'doggo', 'floffer', 'pupper', 'puppo' need to be merged in one column since they holds different values of one variable.


**predict**
- Some names are lowercased. Capitalize'm
- Some rows contains predictions that not mathes with dog breed names. Drop'm

 
<br>

### Tidiness

**df**
- Column 'text' has two types of data, associated with different variables: 1) The text of the tweet, 2) The link to the tweet. The link to the tweet is the same link that stored in the column named ' expanded_urls'. So, the data in column 'text' need to be separated and we need to keep just part 1 - actually the text of the tweet.

**predict**
- All predictions are different level of confident of prediction the value of one variable. We need to form one column with predicted dog  breed name associated with particular tweet_id in descendent priority order: p1>p2>p3

<br>
<a id='Clean_Data'></a>

# Part III - Clean Data
  
    

## Quality

# <br>

### 'df' data frame

In [112]:
df = df_master.copy()

<br>

#### Define

Columns 'in_reply_to_status_id' and 'in_reply_to_user_id' has very little amount of data and make no sense for further analysis. Drop'm

#### Code

Before dropping the columns, lest substract the rows with data in 'in_reply_to_status_id' and see what is in it

In [113]:
#overview with no '.head()'
df[(df['in_reply_to_status_id'].isnull() == False)].head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None


Some rows here are without simbol "@", so we consider tham as original posts. Let's drop other rows

In [114]:
#get list of indexes of rows to drop
list = df[(df['in_reply_to_status_id'].isnull() == False) & (df['text'].str.contains('@'))].index.tolist();

In [115]:
#drop the rows 
df.drop(list, axis = 0, inplace = True)

In [116]:
# now drop the columns
df.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis = 1, inplace = True)

In [117]:
#reset index
df = df.reset_index(drop = True);

#### Test

In [118]:
df.head(1)

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


<br>

#### Define

Data in every cell in column 'timestamp' has '+0000' at the end of the string.

#### Code

In [119]:
#get the instance and the lentgh of string in cell
df.loc[0,'timestamp'], len(df.loc[0,'timestamp'])

('2017-08-01 16:23:56 +0000', 25)

In [120]:
#cut last six elements from the string in each cell in column(space included)
df['timestamp'] = df['timestamp'].str[:19]

#### Test

In [121]:
#check
df.loc[0,'timestamp'], len(df.loc[0,'timestamp'])

('2017-08-01 16:23:56', 19)

<br>

#### Define

Column 'timestamp' in 'object' data type. Convert to 'datetime'

#### Code

In [122]:
#convert data type 
df['timestamp'] = pd.to_datetime(df['timestamp'])

#### Test

In [123]:
df.timestamp[0]

Timestamp('2017-08-01 16:23:56')

In [124]:
df.timestamp.dtype

dtype('<M8[ns]')

<br>

#### Define

Column 'source' has wrong data totally. Drop it

#### Code

In [125]:
df.drop('source', axis = 1, inplace = True)

#### Test

In [126]:
df.head(1)

,tweet_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


<br>

#### Define

Some cells in the column 'text' contains '@' sign with some diffirent chanales names, that associated with retweets. Need to get closer look and figure it out

#### Code

In [127]:
#get the instance 
df.loc[19,'text']

'RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX'

In [128]:
#delete substring  'RT @dog_rates:' from text where it exist
df['text'] = [[x[15:] if 'RT @dog_rates' in x else x] for x in df['text']]
df['text'] = df['text'].str[0]

In [129]:
#check waht is left - overview w/n head()
df[(df['text'].str.contains('@') == True)].head(1)

,tweet_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
31,886054160059072513,2017-07-15 02:45:48,RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,19607400.0,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None


In [130]:
# get instance
df.text[363]

"Meet Tucker. It's his birthday. He's pupset with you because you're too busy playing @GoodDogsGame to celebrate. 13/10 would put down phone https://t.co/vrppizPGdb"

In [131]:
# overview with 'RT @'
df[(df['text'].str.contains('RT @') == True)].head(1)

,tweet_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
31,886054160059072513,2017-07-15 02:45:48,RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,19607400.0,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None


In [132]:
#drop all rows with 'RT @' in 'text' column
df = df[(df['text'].str.contains('RT @') != True)]

#### Test

In [133]:
#check
df.loc[19,'text']

'This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX'

In [134]:
# overview with 'RT @'
df[(df['text'].str.contains('RT @') == True)].head(1)

,tweet_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


<br>

#### Define

Columns 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' are lack of data. Drop'm

#### Code

Before we do that, lets substract the 'df' with data in 'retweeted_status_id' and see what is in it

In [135]:
df[df['retweeted_status_id'].isnull() == False].shape

(156, 14)

In [136]:
# overview
df[df['retweeted_status_id'].isnull() == False].head(1)

,tweet_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,2017-07-21 01:02:36,This is Canela. She attempted some fancy porch...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None


In [137]:
df['text'][160]

'Ohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboyohboy. 10/10 for all (by happytailsresort) https://t.c…'

In [138]:
df['expanded_urls'][160]

'https://twitter.com/dog_rates/status/761672994376806400/video/1'

In [139]:
df[(df['retweeted_status_id'].isnull() == False) & (df['text'].str.contains('@'))]

,tweet_id,timestamp,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
415,820494788566847489,2017-01-15 04:56:16,We are proud to support @LoveYourMelon on thei...,8.203146e+17,4.196984e+09,2017-01-14 17:00:24 +0000,"https://www.loveyourmelon.com/pages/ourstory,h...",14,10,None,None,None,None,None
803,769335591808995329,2016-08-27 00:47:53,Ever seen a dog pet another dog? Both 13/10 tr...,7.069045e+17,4.196984e+09,2016-03-07 18:09:06 +0000,"https://vine.co/v/iXQAm5Lrgrh,https://vine.co/...",13,10,None,None,None,None,None


In [140]:
df.text[415]

'We are proud to support @LoveYourMelon on their mission to put a hat on every kid battling cancer. They are 14/10\n\nhttps://t…'

In [141]:
df.text[803]

'Ever seen a dog pet another dog? Both 13/10 truly an awe-inspiring scene. (Vid by @mdougherty20) https://t.co/3PoKf6cw7f'

In [142]:
#drop columns
df.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis = 1, inplace = True)

#### Test

In [143]:
df.head(2)

,tweet_id,timestamp,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


<br>

#### Define

Duplicated and missing data in column 'expanded_urls'

#### Code

In [144]:
df.shape[0]

2299

In [145]:
df.expanded_urls.nunique()

2194

In [146]:
# count duplicates
df.expanded_urls.duplicated().sum()

104

In [147]:
#count missing data
df['expanded_urls'].isnull().sum()

26

In [148]:
#count rows with url not related to WRD
df[df['expanded_urls'].str.contains('twitter') != True].shape[0]

132

In [149]:
counts_df['expanded_url'].nunique()

1991

Decided to clean this column and get one url associated with WRD

#### Test

In [91]:
df.head(1)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None,None


<br>

#### Define

Column 'rating_numerator' has several outliers, so need to get closer what are they exactly.

#### Code

In [92]:
#get a view of values
df['rating_numerator'].value_counts()

12      543
10      457
11      455
13      337
9       158
8       102
7        55
14       47
5        36
6        32
3        19
4        17
2         9
1         9
0         2
75        2
50        1
20        1
24        1
26        1
44        1
165       1
60        1
80        1
84        1
88        1
143       1
204       1
420       1
1776      1
27        1
45        1
99        1
121       1
144       1
Name: rating_numerator, dtype: int64

In [100]:
#check
ind = df[df['rating_numerator'] == 1776].index[0]

In [101]:
#get tweet_id
id = df.loc[ind, 'tweet_id']

In [111]:
id in counts_df['tweet_id']

False

In [110]:
counts_df.loc[counts_df[counts_df['tweet_id'] == id].index[0],'expanded_url']

IndexError: index 0 is out of bounds for axis 0 with size 0

#### Test

<br>

#### Define

Column 'rating_denominator' - need to figure out the outliers.

#### Code

#### Test

<br>

#### Define

Column 'name' has some wrong data such as 'None' and other words that not suppose to be the names of dogs. Need to figure out how to handle them.

#### Code

#### Test

<br>

#### Define

Columns 'doggo', 'floffer', 'pupper', 'puppo' need to be merged in one column since they holds different values of one variable.

#### Code

In [ ]:
# list of stages for np.select 
stages_list = ['doggo', 'floofer', 'pupper', 'puppo']
# list of conditions for np.select
conditions = [(df['doggo'] == 'doggo'),
             (df['floofer'] == 'floffer'),
             (df['pupper'] == 'pupper'),
             (df['puppo'] == 'puppo')]
#assigning the values for column depending on conditions
df['stages'] = np.select(conditions, stages_list, default = 'the dog is good anyway')

#### Test

# <br>
### 'predict' data frame

<br>

#### Define

Some names are lowercased. Capitalize'm

#### Code

#### Test

<br>

#### Define

Some rows contains predictions that not mathes with dog breed names. Drop'm

#### Code

#### Test

# <br>

## Tidiness

# <br>

## 'df' data frame

<br>

#### Define

Column 'text' has two types of data, associated with different variables: 1) The text of the tweet, 2) The link to the tweet. The link to the tweet is the same link that stored in the column named ' expanded_urls'. So, the data in column 'text' need to be separated and we need to keep just part 1 - actually the text of the tweet.

#### Code

#### Test

# <br>

### 'predict' data frame

<br>

#### Define

All predictions are different level of confident of prediction the value of one variable. We need to form one column with predicted dog breed name associated with particular tweet_id in descendent priority order: p1>p2>p3

#### Code

#### Test